In [2]:
import d2lzh as d2l
import math
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, rnn
import time
import zipfile

ctx = d2l.try_gpu()
print('will use', ctx)

will use cpu(0)


In [3]:
# load dataset
def load_data_jay_lyrics():
    """Load the Jay Chou lyric data set (available in the Chinese book)."""
    with zipfile.ZipFile('d2l-zh/data/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

In [6]:
if vocab_size == 1027:
    print("Jay lyrics dataset is successfully loaded")

Jay lyrics dataset is successfully loaded


In [24]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()

def get_params():
    def _one(shape):
        return nd.random.normal(scale = 0.01, shape = shape, ctx = ctx)
    
    def _three():
        return (_one((num_inputs, num_hiddens)),
               _one((num_hiddens, num_hiddens)),
               nd.zeros(num_hiddens,ctx = ctx))
    
    W_xi, W_hi, b_i = _three() #输入门参数
    W_xf, W_hf, b_f = _three() #遗忘门参数
    W_xo, W_ho, b_o = _three() #输出门参数
    W_xc, W_hc, b_c = _three() #候选细胞门参数
    
    #输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    
    params = [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q]
    for param in params:
        param.attach_grad()
    
    return params

In [25]:
def init_lstm_state(batch_size, num_hiddens, ctx):
    # return initial hidden state and memory cell state
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx),
           nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx))

In [26]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
    (H, C) = state
    outputs = []
    
    for X in inputs:
        I = nd.sigmoid(nd.dot(X, W_xi) + nd.dot(H, W_hi) + b_i)
        F = nd.sigmoid(nd.dot(X, W_xf) + nd.dot(H, W_hf) + b_f)
        O = nd.sigmoid(nd.dot(X, W_xo) + nd.dot(H, W_ho) + b_o)
        C_tilda = nd.tanh(nd.dot(X, W_xc) + nd.dot(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

In [31]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 70, ['分开', '不分开']

In [32]:
d2l.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,
                          vocab_size, ctx, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 50, perplexity 164.921222, time 4.02 sec
 - 分开 我想你我 我不不 我想你的我 我想你你我 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你
 - 不分开 我想你我你的可 我想想你我 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 
epoch 100, perplexity 32.567101, time 4.02 sec
 - 分开 我想你你 我有我想想你 我不不的生笑 爱知好觉 我该了这生我 不知好觉 我不了这生活 我知好觉 我不了这生活 我知好觉 我不了这生活 我知
 - 不分开 我不你的生笑 你想好这生活 我想要你的生笑 爱知你觉 我该了这生我 不知好觉 我不了这生活 我知好觉 我不了这生活 我知好觉 我不了这生活
epoch 150, perplexity 5.397750, time 3.93 sec
 - 分开 你是我 不是是 一九我 一九我的见你的你 我真心你 你已心一个人在抱悲 不不不不了是是她知道 我不再再想想你要 这样躲 说你眼睛看着我 别
 - 不分开 你已的让我听听的怒火 就谢开你开你 让这好的生笑 你 在我胸睡睡 这样 是样的我就听                           
epoch 200, perplexity 1.749087, time 4.14 sec
 - 分开 别悔的我不起 然指林 一什么 我想就这二 对对云人落落寞它的窝人 一括心猫 一根我不抽 菸一好酒 你来一日蛛 白色苦烛 温家了空屋 白色蜡
 - 不分开 我已能这样 我想要这样 我爱经这节奏 后知后觉 你过了一个我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该
epoch 250, perplexity 1.219279, time 4.15 sec
 - 分开 问候我 谁是神枪手 巫师 他念念 有词的 对酋长下诅咒 还我骷髅头 这故事 告诉我 印地安的传说 还真是 瞎透了 什么都有 沙漠之中怎么会
 - 不分开 我已经这天我 想知和你担忧 唱着歌一居都猜到我都在的感受 河边的风 在吹着头发飘动 牵着你的手 一阵莫名感动 我想带你 回我的外婆家 一起
